In [1]:
import pymysql
conn = pymysql.connect(host='localhost',port=3306,db='lolgamequiz',user='root',passwd='root',charset='utf8')
def sele(sql):
    cur = conn.cursor()
    cur.execute(sql)
    result = list(cur.fetchall())
    result = [list(r) for r in result]
    conn.commit()
    cur.close()
    
    return result


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [22]:
sql = 'select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2020\
UNION ALL select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2019'
data = sele(sql)
results = pd.DataFrame(data, columns=['date', 'processor','dayNum','dayOrder', 'teamA', 'teamB', 'scoreA', 'scoreB', 'fullTeamA', 'fullTeamB'])
print(results)

sql = 'select teamName, rank, region from rank2020 where region <> "Play-In"\
UNION ALL select teamName, rank, region from rank2019 where region <> "Play-In"'
data = sele(sql)
rank = pd.DataFrame(data, columns=['teamName', 'rank', 'region'])
print(rank)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ALL select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullT' at line 1")

In [23]:
# clean data
# cover data type
results = results.astype({'date':'datetime64', 'scoreA':'int64', 'scoreB':'int64'})
# fix team name
results['teamA'] = results['teamA'].apply(lambda x: x.upper())
results['teamB'] = results['teamB'].apply(lambda x: x.upper())
results['fullTeamA'] = results['fullTeamA'].apply(lambda x: x.upper())
results['fullTeamB'] = results['fullTeamB'].apply(lambda x: x.upper())
# sort
results = results.sort_values(by=['date','dayNum','dayOrder'])
results = results.reset_index()
results = results.drop(['index'], axis=1)
print(results)


####################
# rank
# region
rank = rank.astype({'rank':'int64'})
rank['teamName'] = rank['teamName'].apply(lambda x: x.upper())
print(rank)

         date   processor               dayNum dayOrder teamA teamB  scoreA  \
0  2020-09-25     Play-In                Day 1        1   MAD   ITZ       1   
1  2020-09-25     Play-In                Day 1        2   PSG    R7       1   
2  2020-09-25     Play-In                Day 1        3   LGC   ITZ       1   
3  2020-09-25     Play-In                Day 1        4   LGD   PSG       0   
4  2020-09-25     Play-In                Day 1        5    TL   MAD       1   
5  2020-09-26     Play-In                Day 2        1    R7    V3       0   
6  2020-09-26     Play-In                Day 2        2   ITZ   SUP       0   
7  2020-09-26     Play-In                Day 2        3    V3   UOL       0   
8  2020-09-26     Play-In                Day 2        4   LGD    R7       0   
9  2020-09-26     Play-In                Day 2        5   UOL   PSG       1   
10 2020-09-26     Play-In                Day 2        6   SUP   MAD       1   
11 2020-09-26     Play-In                Day 2      

In [24]:

gy = rank['region'].groupby(rank['region'])
print(gy.count())

region
BR     1
CIS    2
CN     5
EU     4
JP     1
KR     3
LAT    1
NA     4
OCE    1
PCS    3
TR     1
Name: region, dtype: int64


In [25]:
# won
results['Awon'] = False
results['Bwon'] = False
results['Awon'] = results['scoreA']>results['scoreB']
results['Bwon'] = results['scoreA']<results['scoreB']
results

,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,...,BisSEA,BisTR,BisVN,rankA,rankB,Alastwon,Blastwon,lastAwon,lastBwon,BO1
0,2020-09-25,Play-In,Day 1,1,MAD,ITZ,1,0,MAD LIONS,INTZ,...,False,False,False,4,1,False,False,False,False,True
1,2020-09-25,Play-In,Day 1,2,PSG,R7,1,0,PSG TALON,RAINBOW7,...,False,False,False,2,1,False,False,False,False,True
2,2020-09-25,Play-In,Day 1,3,LGC,ITZ,1,0,LEGACY ESPORTS,INTZ,...,False,False,False,1,1,False,False,False,False,True
3,2020-09-25,Play-In,Day 1,4,LGD,PSG,0,1,LGD GAMING,PSG TALON,...,False,False,False,4,2,False,True,False,False,True
4,2020-09-25,Play-In,Day 1,5,TL,MAD,1,0,TEAM LIQUID,MAD LIONS,...,False,False,False,3,4,False,True,False,False,True
5,2020-09-26,Play-In,Day 2,1,R7,V3,0,1,RAINBOW7,V3 ESPORTS,...,False,False,False,1,1,False,False,False,False,True
6,2020-09-26,Play-In,Day 2,2,ITZ,SUP,0,1,INTZ,SUPERMASSIVE ESPORTS,...,False,True,False,1,1,False,False,False,False,True
7,2020-09-26,Play-In,Day 2,3,V3,UOL,0,1,V3 ESPORTS,UNICORNS OF LOVE.CIS,...,False,False,False,1,1,True,False,False,False,True
8,2020-09-26,Play-In,Day 2,4,LGD,R7,0,1,LGD GAMING,RAINBOW7,...,False,False,False,4,1,False,False,False,False,True
9,2020-09-26,Play-In,Day 2,5,UOL,PSG,1,0,UNICORNS OF LOVE.CIS,PSG TALON,...,False,False,False,1,2,True,True,False,False,True


In [26]:

############################################
# region
results['regionA'] = ''
results['regionB'] = ''

results['AisBR'] = False
results['AisCIS'] = False
results['AisCN'] = False
results['AisEU'] = False
results['AisJP'] = False
results['AisKR'] = False
results['AisLAT'] = False
results['AisLMS'] = False
results['AisNA'] = False
results['AisOCE'] = False
results['AisSEA'] = False
results['AisTR'] = False
results['AisVN'] = False

results['BisBR'] = False
results['BisCIS'] = False
results['BisCN'] = False
results['BisEU'] = False
results['BisJP'] = False
results['BisKR'] = False
results['BisLAT'] = False
results['BisLMS'] = False
results['BisNA'] = False
results['BisOCE'] = False
results['BisSEA'] = False
results['BisTR'] = False
results['BisVN'] = False

for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank[rank['teamName']==teamA]
    if not regionA.empty:
        regionA = regionA['region'].values[0]
        results.loc[index,'regionA'] = regionA
        if regionA == 'BR':
            results.loc[index,'AisBR'] = True
        elif regionA == 'CIS':
            results.loc[index,'AisCIS'] = True
        elif regionA == 'CN':
            results.loc[index,'AisCN'] = True
        elif regionA == 'EU':
            results.loc[index,'AisEU'] = True
        elif regionA == 'JP':
            results.loc[index,'AisJP'] = True
        elif regionA == 'KR':
            results.loc[index,'AisKR'] = True
        elif regionA == 'LAT':
            results.loc[index,'AisLAT'] = True
        elif regionA == 'LMS':
            results.loc[index,'AisLMS'] = True
        elif regionA == 'NA':
            results.loc[index,'AisNA'] = True
        elif regionA == 'OCE':
            results.loc[index,'AisOCE'] = True
        elif regionA == 'SEA':
            results.loc[index,'AisSEA'] = True
        elif regionA == 'TR':
            results.loc[index,'AisTR'] = True
        elif regionA == 'VN':
            results.loc[index,'AisVN'] = True
        
    regionB = rank[rank['teamName']==teamB]
    if not regionB.empty:
        regionB = regionB['region'].values[0]
        results.loc[index,'regionB'] = regionB
        if regionB == 'BR':
            results.loc[index,'BisBR'] = True
        elif regionB == 'CIS':
            results.loc[index,'BisCIS'] = True
        elif regionB == 'CN':
            results.loc[index,'BisCN'] = True
        elif regionB == 'EU':
            results.loc[index,'BisEU'] = True
        elif regionB == 'JP':
            results.loc[index,'BisJP'] = True
        elif regionB == 'KR':
            results.loc[index,'BisKR'] = True
        elif regionB == 'LAT':
            results.loc[index,'BisLAT'] = True
        elif regionB == 'LMS':
            results.loc[index,'BisLMS'] = True
        elif regionB == 'NA':
            results.loc[index,'BisNA'] = True
        elif regionB == 'OCE':
            results.loc[index,'BisOCE'] = True
        elif regionB == 'SEA':
            results.loc[index,'BisSEA'] = True
        elif regionB == 'TR':
            results.loc[index,'BisTR'] = True
        elif regionB == 'VN':
            results.loc[index,'BisVN'] = True
############################################
# region/rank

results['rankA'] = ''
results['rankB'] = ''

for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank.loc[rank[rank['teamName']==teamA].index,'region']
    regionB = rank.loc[rank[rank['teamName']==teamB].index,'region']
    
    rankA = rank.loc[rank[rank['teamName']==teamA].index,'rank']
    rankB = rank.loc[rank[rank['teamName']==teamB].index,'rank']
    
    if not regionA.empty:
        #results.loc[index, 'regionA'] = regionA.values[0]
        results.loc[index, 'rankA'] = rankA.values[0]
    if not regionB.empty:
        #results.loc[index, 'regionB'] = regionB.values[0]
        results.loc[index, 'rankB'] = rankB.values[0]

##################################
# last won
results['Alastwon'] = False
results['Blastwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        won_last[teamA] = row['Awon']
        won_last[teamB] = row['Bwon']
        continue

    results.loc[index, 'Alastwon'] = won_last[teamA]
    results.loc[index, 'Blastwon'] = won_last[teamB]
    won_last[teamA] = row['Awon']
    won_last[teamB] = row['Bwon']

##################################
# last who won
results['lastAwon'] = False
results['lastBwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        continue
    for index1, row1 in results[:index].iterrows():
        if row1['teamA'] == teamA and row1['teamB'] == teamB:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Awon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Bwon']
        elif row1['teamA'] == teamB and row1['teamB'] == teamA:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Bwon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Awon']

##################################
# BO
def bo(score):
    max_score = max(score['scoreA'], score['scoreB'])
    if max_score == 1:
        return True
    elif max_score == 3:
        return False
    else:
        return False
        
results['BO1'] = results[['scoreA','scoreB']].apply(bo, axis=1)

results



,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,...,BisSEA,BisTR,BisVN,rankA,rankB,Alastwon,Blastwon,lastAwon,lastBwon,BO1
0,2020-09-25,Play-In,Day 1,1,MAD,ITZ,1,0,MAD LIONS,INTZ,...,False,False,False,4,1,False,False,False,False,True
1,2020-09-25,Play-In,Day 1,2,PSG,R7,1,0,PSG TALON,RAINBOW7,...,False,False,False,2,1,False,False,False,False,True
2,2020-09-25,Play-In,Day 1,3,LGC,ITZ,1,0,LEGACY ESPORTS,INTZ,...,False,False,False,1,1,False,False,False,False,True
3,2020-09-25,Play-In,Day 1,4,LGD,PSG,0,1,LGD GAMING,PSG TALON,...,False,False,False,4,2,False,True,False,False,True
4,2020-09-25,Play-In,Day 1,5,TL,MAD,1,0,TEAM LIQUID,MAD LIONS,...,False,False,False,3,4,False,True,False,False,True
5,2020-09-26,Play-In,Day 2,1,R7,V3,0,1,RAINBOW7,V3 ESPORTS,...,False,False,False,1,1,False,False,False,False,True
6,2020-09-26,Play-In,Day 2,2,ITZ,SUP,0,1,INTZ,SUPERMASSIVE ESPORTS,...,False,True,False,1,1,False,False,False,False,True
7,2020-09-26,Play-In,Day 2,3,V3,UOL,0,1,V3 ESPORTS,UNICORNS OF LOVE.CIS,...,False,False,False,1,1,True,False,False,False,True
8,2020-09-26,Play-In,Day 2,4,LGD,R7,0,1,LGD GAMING,RAINBOW7,...,False,False,False,4,1,False,False,False,False,True
9,2020-09-26,Play-In,Day 2,5,UOL,PSG,1,0,UNICORNS OF LOVE.CIS,PSG TALON,...,False,False,False,1,2,True,True,False,False,True


In [43]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit_transform(results['regionA'].values)
regionA = encoding.fit_transform(results['regionA'].values)
regionB = encoding.fit_transform(results['regionB'].values)

region = np.vstack([regionA, regionB]).T

from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
r = onehot.fit_transform(region).todense()
df = pd.DataFrame(r)

index = results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index

#X_previouswins = results.loc[]
df1 = df.loc[index,:]

#X_previouswins = X_previouswins.join(df)
X_previouswins = df1.values


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
forest = joblib.load('forest.m')


forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)



y_true = results.loc[results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index,['Awon']].values
forest.fit(X_previouswins,y_true)
y_pre = forest.predict(X_previouswins)
y_pre

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


array([False,  True,  True,  True, False, False,  True,  True, False,
       False,  True, False, False, False,  True, False,  True,  True,
        True, False,  True,  True, False, False, False, False,  True,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False,  True,  True,  True, False,
       False, False, False, False,  True,  True, False, False,  True,
        True, False, False,  True,  True,  True,  True, False, False,
       False, False,  True,  True, False,  True, False, False, False,
        True,  True, False, False,  True, False, False, False,  True,
        True, False, False,  True,  True, False,  True])

In [45]:
success = 0
for index in range(y_pre.shape[0]):
    Awon_p = y_pre[index]
    Awon_t = y_true[index]
    if Awon_p == Awon_t:
        success += 1
print('score: {:.2f}%'.format((success/y_pre.shape[0])*100))

score: 73.58%


In [46]:

results_pre = results.loc[results[(results['scoreA']+results['scoreB']==0) & (results['processor']=='Main_Event')].index,['date','dayNum','dayOrder','teamA','teamB', 'Awon', 'Bwon']]
results_pre = results_pre.reset_index()
index = results[(results['scoreA']+results['scoreB']==0) & (results['processor']=='Main_Event')].index
df1 = df.loc[index,:]
X_previouswins = df1.values

y_pre = forest.predict(X_previouswins)

for index in range(y_pre.shape[0]):
    results_pre.loc[index,'Awon'] = y_pre[index]
    results_pre.loc[index,'Bwon'] = not y_pre[index]

results_pre

,index,date,dayNum,dayOrder,teamA,teamB,Awon,Bwon
0,165,2020-10-15,Quarterfinals,4,DWG,DRX,False,True
1,166,2020-10-16,Quarterfinals,2,SN,JDG,True,False
2,167,2020-10-17,Quarterfinals,1,TES,FNC,True,False
3,168,2020-10-18,Quarterfinals,3,GEN,G2,True,False


In [21]:
from sklearn.externals import joblib
forest = joblib.load('forest.m')
X_previouswins = results.loc[results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index,
                             ['AisBR','AisCIS','AisCN','AisEU','AisJP','AisKR','AisLAT','AisLMS','AisNA','AisOCE','AisSEA','AisTR','AisVN',      
                          'BisBR','BisCIS','BisCN','BisEU','BisJP','BisKR','BisLAT','BisLMS','BisNA','BisOCE','BisSEA','BisTR','BisVN'
                          ,'BO1','lastAwon']].values
y_true = results.loc[results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index,['Awon']].values

print(X_previouswins)

y_pre = forest.predict(X_previouswins)
success = 0
for index in range(y_pre.shape[0]):
    Awon_p = y_pre[index]
    Awon_t = y_true[index]
    if Awon_p == Awon_t:
        success += 1
print('score: {:.2f}%'.format((success/y_pre.shape[0])*100))


[[False False False ... False  True False]
 [False  True False ... False  True False]
 [False False False ... False  True False]
 ...
 [False False  True ... False  True  True]
 [False  True False ... False  True False]
 [False False False ... False  True False]]
score: 67.35%


In [14]:
from sklearn.model_selection import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_previouswins, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

NameError: name 'DecisionTreeClassifier' is not defined

In [174]:
forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)
scores = cross_val_score(forest, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))
from sklearn.externals import joblib
joblib.dump(forest, 'forest.m')
joblib.load('forest.m')

score: 72.94%


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)